In [2]:
from sklearn.model_selection import train_test_split
import networkx as nx
import pandas as pd
import numpy as np
import random
%matplotlib notebook

# Loading the csv file containing the network

In [3]:
csvfile = open('graphnodes.csv', 'r')
next(csvfile, None)  # skip the first line in the graphnodes.csv file  
Graph = nx.Graph()  # create networkx graph object
G = nx.parse_edgelist(csvfile, delimiter=',', create_using=Graph)

# Calculating the scores of each non-connected edge

## Common Neighbor Index

In [4]:
def common_neighbor(G):
    comm_neigh_scores = {}
    
    edges = np.array(G.edges())
    train_set, test_set = train_test_split(edges, test_size=0.3, shuffle=False)  # split the nodes into train test (70%) and test set (30%)
    non_conn_edges = np.array(list(nx.non_edges(G)))  # non-connected edges in array form
    
    # prediction set (Non-connected edges + Test set)
    pred_set = np.concatenate((non_conn_edges, test_set))  # joining set of non-connected edges to the training set for prediction
    
    for u,v in pred_set:                             # u and v are any two non-connected nodes
        u_neighbors = set(G.neighbors(u))                   # finding the neighbors of u
        v_neighbors = set(G.neighbors(v))                   # finding the neighbors of v
        score_uv = len(u_neighbors & v_neighbors)           # length of the set containing the intersections of neighbors
        comm_neigh_scores[(u,v)] = score_uv  # adds the node and its corresponding score in the comm_neigh_scores dictionary
        
    return comm_neigh_scores

## Katz Index

In [5]:
def katz(G):
    katz_scores = {}
    
    """Calculating the beta value. The beta value is chosen such that it is less than the 
    reciprocal of the maximum eigenvalue of the adjacency matrix of the network.
    This is done to ensure convergence of the katz algorithm."""
    
    adj_matrix = nx.to_numpy_matrix(G)  # generates the adjacency matrix of the network
    eig_vals = np.linalg.eigvals(adj_matrix)  # the eigenvalues of the adjacency matrix of the network
    beta_ceil = 1 / max(eig_vals)  #ceiling value for beta
    beta = random.uniform(0, beta_ceil)
    
    edges = np.array(G.edges())  # all connected edges of the network
    train_set, test_set = train_test_split(edges, test_size=0.3, shuffle=False)  # split the nodes into train test (70%) and test set (30%)
    non_conn_edges = np.array(list(nx.non_edges(G)))  # non-connected edges in array form
    
    # prediction set (Non-connected edges + Test set)
    pred_set = np.concatenate((non_conn_edges, test_set))  # joining set of non-connected edges to the training set for prediction
    
    def path_length(n):
        all_paths = nx.all_simple_paths(G, u, v)
        return [path for path in all_paths if len(path) == n]
        
    for u,v in pred_set:
        # bypassing error of finding the max of an empty list
        try:  
            # longest path length of paths connecting u and v
            max_length = max([len(path) for path in nx.all_simple_paths(G, u, v)]) 
        except ValueError as e:
            print('Sorry', e)
            continue

        """" counts path lengths of length l and stores them in a path_length_counts
        such that index 1 holds the count of paths of length 1
        and index 2 holds the count of paths of length 2, in that order..."""
        
        path_length_counts = [len(path_length(l)) for l in range(1, max_length+1)]
        np_path_length_counts = np.array(path_length_counts)  #converts np_path_length_counts to numpy array
        beta_powers = [beta ** power for power in range(1, max_length+1)] # list of beta value raised to powers 1,2,..,max_length
        np_beta = np.array(beta_powers) # converts array of beta in previous line to numpy array to facilitate vectorization
    
        score_uv = sum(np_beta * np_path_length_counts) # the katz score for nodes u,v
        katz_scores[(u,v)] = score_uv  # adds the node and its corresponding score in the katz_scores dictionary
    
    return katz_scores

# Calculating the precision of the link prediction indices using the Area Under the receiver Characteristic curve (AUC) method.

In [7]:
def auc(G):  # index_type is either common_neighbor, katz, or cn_katz
    
    index_type = input("Please enter the type of index (you may use commnon_neighbor, katz, or cn_katz: )")
    acceptable_index_types = ['common_neighbor', 'katz', 'cn_katz']
    
    if index_type not in acceptable_index_types:
        return(print('\nSorry, you did not enter the right name.\n\nUse either common_neighbor, katz, or cn_katz'))
    
    edges = np.array(G.edges())
    train_set, test_set = train_test_split(edges, test_size=0.3, shuffle=False)  # split the nodes into train test (70%) and test set (30%)
    train_set_tuple_entries = list(map(tuple, train_set))  # converts train_set to list with tuple entries
    test_set_tuple_entries = list(map(tuple, test_set))  # converts test_set to list with tuple entries
    
    # prediction set (Non-connected edges + Test set)
    non_conn_edges = np.array(list(nx.non_edges(G)))  # non-connected edges in array form
    non_conn_edges_tuple_entries = list(map(tuple, non_conn_edges))
    
    n = len(test_set) * len(non_conn_edges)  # n is the number of comparisons
    n1 = 0  # n1 is the number of times the missing links (nodes in test_set) have higher score than the non-existent links (nodes on pred_test)
    n2 = 0  # n2 is the number of times the missing links (nodes in test_set) have the same score as the non-existent links (nodes on pred_test)

    for i in range(n):
        if index_type == 'common_neighbor':
            for test_node in test_set_tuple_entries:
                for non_conn_node in non_conn_edges_tuple_entries:
                    if common_neighbor(G)[test_node] > common_neighbor(G)[non_conn_node]:
                        n1 += 1
                    elif common_neighbor(G)[test_node] == common_neighbor(G)[non_conn_node]:
                        n2 += 1

        elif index_type == 'katz':
            for test_node in test_set_tuple_entries:
                for non_conn_node in non_conn_edges_tuple_entries:
                    if katz(G)[test_node] > katz(G)[non_conn_node]:
                        n1 += 1
                    elif katz(G)[test_node] == katz(G)[non_conn_node]:
                        n2 += 1

        else:
            for test_node in test_set_tuple_entries:
                for non_conn_node in non_conn_edges_tuple_entries:
                    if cn_katz(G)[test_node] > cn_katz(G)[non_conn_node]:
                        n1 += 1
                    elif cn_katz(G)[test_node] == cn_katz(G)[non_conn_node]:
                        n2 += 1
    
    auc = (n1 + 0.5*n2) / float(n)  # the AUC value
    return auc

# Running the algorithms

## Common Neighbor

In [ ]:
common_neighbor(G)

## Katz

In [ ]:
katz(G)

## AUC

In [ ]:
auc(G)

### A plot of the graph

In [ ]:
# nx.draw_networkx(G, with_labels = True, node_size = 100)